In [1]:
!pip install tensorflow==2.0
!pip install unidecode

     |████████████████████████████████| 86.3MB 50kB/s 
     |████████████████████████████████| 450kB 53.6MB/s 
     |████████████████████████████████| 3.8MB 42.6MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 245kB 2.8MB/s 


In [2]:
%tensorflow_version 2.x
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import pickle
import csv
import random
import tensorflow as tf
from tensorflow.keras import Sequential
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Embedding, Reshape
print(tf.__version__)


2.0.0


In [3]:
url="https://raw.githubusercontent.com/tlkh/text-emotion-classification/master/data.csv"
import urllib.request
urllib.request.urlretrieve(url, "data.csv")
sentence=[]
sentiment=[]
texts=[]
with open("data.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        texts.append(row)
print(texts[12])
random.shuffle(texts)
for row in texts:
        sentence.append(row[0])
        sentiment.append(int(row[1]))
# neutral, happy, sad, anger, hate.


['angry guy screws his gf in very rude manner', '4']


In [0]:
# import tensorflow_hub as hub
# embed = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1") 
# embeddings = embed(sentence)
# sent_array = embeddings.numpy()
# print(embeddings[7])
# print(embeddings.shape)


In [4]:
import tensorflow_datasets as tfds
import unidecode
token =  tfds.features.text.Tokenizer()
vocab = [token.tokenize(x) for x in sentence]
vocab = list(set([y for x in vocab for y in x]))
encoder = tfds.features.text.SubwordTextEncoder(vocab_list=vocab)
X_train = np.array([encoder.encode(x) for x in sentence])
print(X_train.shape)
print(X_train[:5])
print(len(np.unique(X_train)))

(47288,)
[list([4405, 34393, 8201, 34393, 21593, 34393, 999, 34393, 21492, 34393, 9455, 34393, 2304])
 list([10320, 34393, 12435, 34393, 2464, 34393, 21087, 34393, 11547, 34393, 722, 34393, 7740, 34393, 20358, 34393, 29736, 34393, 7740, 34393, 3499, 34393, 134, 34393, 25798, 34393, 1756, 34393, 10488, 34393, 34018, 34393, 7740, 34393, 23791, 34393, 6974, 34393, 5088, 34393, 24688, 34393, 17079])
 list([28980, 34393, 999, 34393, 21960, 34393, 26504, 34393, 3068, 34393, 20392, 34393, 19638, 34393, 13189, 34393, 5088, 34393, 22466, 34393, 21674, 34393, 18355, 34393, 999, 34393, 1997, 34393, 18355, 34393, 32763, 34393, 5088, 34393, 6246, 34393, 18505, 34393, 29121, 34393, 28980, 34393, 12038, 34393, 4405, 34393, 3528, 34393, 5802])
 list([18132, 34393, 31763, 34393, 22916, 34393, 29106, 34393, 28466, 34393, 24416, 34393, 12675, 34393, 19631, 34393, 14223, 34393, 12176, 34393, 23401, 34393, 33195])
 list([26504, 34393, 8231, 34393, 105, 34393, 10388, 34393, 999, 34393, 11026, 34393, 999, 34

In [6]:
from tensorflow.keras.preprocessing import sequence
X_train = sequence.pad_sequences(X_train, maxlen=80)
max_features = 60000

tf.keras.backend.clear_session()  # For easy reset of notebook state.

model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9999, amsgrad=True),
              metrics=['accuracy'])
model.fit(X_train, np.array(sentiment), batch_size=64, epochs=5, validation_split=0.2, verbose=1)

Train on 37830 samples, validate on 9458 samples
Epoch 1/5
37830/37830 [==============================] - 130s 3ms/sample - loss: 1.1600 - accuracy: 0.4861 - val_loss: 0.9915 - val_accuracy: 0.5921
Epoch 2/5
37830/37830 [==============================] - 127s 3ms/sample - loss: 0.8960 - accuracy: 0.6438 - val_loss: 0.9500 - val_accuracy: 0.5948
Epoch 3/5
37830/37830 [==============================] - 127s 3ms/sample - loss: 0.7427 - accuracy: 0.7162 - val_loss: 1.0124 - val_accuracy: 0.5888
Epoch 4/5
37830/37830 [==============================] - 127s 3ms/sample - loss: 0.6109 - accuracy: 0.7716 - val_loss: 1.1216 - val_accuracy: 0.5737
Epoch 5/5
37830/37830 [==============================] - 127s 3ms/sample - loss: 0.5015 - accuracy: 0.8121 - val_loss: 1.2538 - val_accuracy: 0.5674


In [7]:
output = ["neutral", "happy", "sad", "anger", "hate"]
print("i m still working",output[np.argmax(model.predict(sequence.pad_sequences([encoder.encode("i m still working")])))])
print("my computer is slow",output[np.argmax(model.predict(sequence.pad_sequences([encoder.encode("my computer is slow")])))])
print("fuck you",output[np.argmax(model.predict(sequence.pad_sequences([encoder.encode("fuck you")])))])
print("shit",output[np.argmax(model.predict(sequence.pad_sequences([encoder.encode("shit")])))])
print("how dare you",output[np.argmax(model.predict(sequence.pad_sequences([encoder.encode("how dare you")])))])


i m still working sad
my computer is slow sad
fuck you anger
shit anger
how dare you sad


In [0]:
import matplotlib.pyplot as plt
plt.hist(np.array(sentiment))
plt.show()